# cellline_data_reader 代码练习

In [1]:
import os
os.chdir('D:/github_project/BioXNet')
os.getcwd()

'D:\\github_project\\BioXNet'

In [2]:
import sys
sys.path.append('D:/github_project/BioXNet/data_loader')
sys.path

['d:\\github_project\\BioXNet\\coding_practice',
 'd:\\anaconda\\envs\\bioxnet\\python311.zip',
 'd:\\anaconda\\envs\\bioxnet\\DLLs',
 'd:\\anaconda\\envs\\bioxnet\\Lib',
 'd:\\anaconda\\envs\\bioxnet',
 '',
 'd:\\anaconda\\envs\\bioxnet\\Lib\\site-packages',
 'd:\\anaconda\\envs\\bioxnet\\Lib\\site-packages\\win32',
 'd:\\anaconda\\envs\\bioxnet\\Lib\\site-packages\\win32\\lib',
 'd:\\anaconda\\envs\\bioxnet\\Lib\\site-packages\\Pythonwin',
 'D:/github_project/BioXNet/data_loader']

In [3]:
import pandas as pd
import numpy as np
import json
from os.path import exists
from pathlib import Path
from parse_config import ConfigParser
from sklearn.model_selection import train_test_split, KFold

## 读取 .json 文件

In [4]:
# 读取 .json 文件保存为 config 对象
with open('D:\github_project\BioXNet\config\gdsc.json') as f:
    config = json.load(f)

print(type(config))
config

<class 'dict'>


{'name': 'BioXNet_GDSC',
 'n_gpu': 1,
 'pretrain': None,
 'transfer_directly': False,
 'arch': {'type': 'BioXNet',
  'args': {'n_hidden_layers': 5,
   'direction': 'root_to_leaf',
   'dropout_list': [0.5, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
   'sparse': True,
   'add_unk_genes': False,
   'batch_normal': True,
   'use_bias': True,
   'attention': True,
   'sparse_first_layer': True,
   'class_num': None,
   'n_outputs': 6,
   'shuffle_genes': False}},
 'data_loader': {'type': 'BioXNetDataLoader',
  'args': {'data_source_list': ['GDSC'],
   'input_data_order': ['drug_target', 'mutation', 'cnv', 'methylation'],
   'batch_size': 1024,
   'seed': 0,
   'shuffle': True,
   'num_workers': 8,
   'eval_dataset': True,
   'pre_params': None,
   'gene_intersection': True,
   'params': {'data_dir': 'data/',
    'remove_response_outlier': True,
    'mut_binary': True,
    'selected_genes': None,
    'combine_type': 'intersection',
    'use_coding_genes_only': True}}},
 'optimizer': {'type': 'Adam',
  '

设置 CellLineBaseData() 参数

In [5]:
params = config['data_loader']['args']['params']
params

{'data_dir': 'data/',
 'remove_response_outlier': True,
 'mut_binary': True,
 'selected_genes': None,
 'combine_type': 'intersection',
 'use_coding_genes_only': True}

In [6]:
data_path = Path(params['data_dir'])
data_list = config['data_loader']['args']['data_source_list']

config = ConfigParser(config)
logger = config.get_logger('train')

remove_response_outlier = params['remove_response_outlier']
mut_binry = params['mut_binary']
selected_genes = params['selected_genes']
combine_type = params['combine_type']
use_coding_genes_only = params['use_coding_genes_only']

data_folder_name = '_'.join(data_list)
prepared_path = Path(params['data_dir']).joinpath(data_folder_name, 'prepared')
processed_path = Path(params['data_dir']).joinpath(data_folder_name, 'processed')

## 导入数据

In [7]:
if exists(processed_path.joinpath('cnv_allgenes.csv')) and \
    exists(processed_path.joinpath('methylation_allgenes.csv')) and \
        exists(processed_path.joinpath('mut_allgenes.csv')) and \
            exists(processed_path.joinpath('drug_target_allgenes.csv')) and \
                exists(processed_path.joinpath('response.csv')):
    logger.info('Loading processed data...')
    recreate = False
    response_df = pd.read_csv(processed_path.joinpath('response.csv'))
    cnv_allgenes_df = pd.read_csv(processed_path.joinpath('cnv_allgenes.csv'), index_col=0)
    methylation_allgenes_df = pd.read_csv(processed_path.joinpath('methylation_allgenes.csv'), index_col=0)
    mut_allgenes_df = pd.read_csv(processed_path.joinpath('mut_allgenes.csv'), index_col=0)
    drug_target_allgenes_df = pd.read_csv(processed_path.joinpath('drug_target_allgenes.csv'), index_col=0)
else:
    logger.info('Processing data...\n')
    recreate = True
    if not exists(processed_path):
        os.makedirs(processed_path)

Loading processed data...


In [8]:
'''
    CellLine
    DrugName
    pIC50
'''
response_df

,CellLine,DrugName,pIC50
0,2313287,ve821,-2.323267
1,2313287,sb505124,-4.696090
2,2313287,nvp-adw742,-2.289179
3,2313287,lfm-a13,-4.194402
4,2313287,tipifarnib,-1.783170
...,...,...,...
135348,ZR7530,dacarbazine,-6.984202
135349,ZR7530,alisertib,-6.398641
135350,ZR7530,serdemetan,-4.534220
135351,ZR7530,pfi-1,-5.755940


In [9]:
'''
    行：Cell Line ID
    列：基因
    数值：
'''
cnv_allgenes_df

,A1BG,A1CF,A2M,A2ML1,A4GALT,A4GNT,AAAS,AACS,AADAC,AADACL2,...,ZSWIM6,ZSWIM7,ZW10,ZWILCH,ZWINT,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1
MKN7,-1.0,0.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ML1,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,...,0.0,-1.0,-1.0,0.0,0.0,-1.0,0.0,0.0,0.0,-1.0
NCIH727,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
KYSE150,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
CFPAC1,0.0,0.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HCC1806,2.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,...,0.0,1.0,0.0,-1.0,1.0,0.0,0.0,0.0,-1.0,1.0
TGBC11TKB,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
JHOS4,0.0,0.0,0.0,0.0,-1.0,1.0,0.0,0.0,1.0,1.0,...,-1.0,-1.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,-1.0
HDQP1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
'''
    行：Cell Line ID
    列：基因
    数值：
'''
methylation_allgenes_df

,A1BG,A1CF,A2M,A2ML1,A4GALT,A4GNT,AAAS,AACS,AADAC,AADACL2,...,ZSWIM6,ZSWIM7,ZW10,ZWILCH,ZWINT,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1
MKN7,0.710512,0.227708,0.618281,0.346031,0.436752,0.614755,0.054102,0.743060,0.572642,0.927505,...,0.459163,0.499197,0.119199,0.973012,0.026057,0.328689,0.478661,0.554666,0.406558,0.861640
NCIH1993,0.589580,0.105943,0.672902,0.621886,0.460210,0.715865,0.048095,0.690342,0.935156,0.981371,...,0.489868,0.570094,0.124838,0.953271,0.022247,0.369295,0.568798,0.535669,0.412937,0.751553
ML1,0.688824,0.486229,0.652128,0.827150,0.445521,0.767321,0.047809,0.730234,0.794626,0.904699,...,0.432948,0.458183,0.138115,0.109570,0.028110,0.358427,0.611773,0.463618,0.344684,0.861372
KYSE220,0.805510,0.476164,0.892299,0.830828,0.496052,0.899470,0.048357,0.746053,0.970037,0.969356,...,0.482890,0.395533,0.114802,0.670497,0.023519,0.336920,0.518939,0.544510,0.419539,0.864015
KYSE150,0.803872,0.596404,0.742780,0.813555,0.429735,0.656537,0.043895,0.762867,0.626613,0.413047,...,0.484634,0.448256,0.127416,0.704510,0.020847,0.459663,0.914930,0.604653,0.424840,0.857296
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
JHOS4,0.714216,0.517499,0.870170,0.844866,0.516090,0.847891,0.050675,0.729433,0.928493,0.965618,...,0.486940,0.461535,0.131745,0.292033,0.029587,0.362261,0.565577,0.473536,0.409063,0.802106
TGBC11TKB,0.524957,0.116692,0.343562,0.202335,0.559644,0.464380,0.040768,0.660311,0.306231,0.393773,...,0.430061,0.389831,0.122114,0.252118,0.023759,0.362111,0.644836,0.523222,0.319536,0.666475
HSC39,0.581953,0.120465,0.251691,0.677080,0.514372,0.314496,0.069234,0.671043,0.933540,0.944038,...,0.492201,0.466763,0.123796,0.494649,0.029787,0.332064,0.847463,0.523201,0.384977,0.726546
HDQP1,0.707415,0.154301,0.771211,0.576787,0.452187,0.840602,0.044525,0.740258,0.891500,0.961499,...,0.470597,0.472687,0.126146,0.398706,0.023687,0.354041,0.486584,0.548996,0.421067,0.831403


In [11]:
'''
    行：Cell Line ID
    列：基因
    数值：0/1，Cell Line 对应的基因突变
'''
mut_allgenes_df

,A1BG,A1CF,A2M,A2ML1,A4GALT,A4GNT,AAAS,AACS,AADAC,AADACL2,...,ZSWIM6,ZSWIM7,ZW10,ZWILCH,ZWINT,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1
22RV1,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2313287,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
42MGBA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
451LU,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5637,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YH13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
YKG1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
YMB1E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
YT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
'''
    行：药物
    列：基因
    数值：0/1，药物对应的 target gene
'''
drug_target_allgenes_df

,A1BG,A1CF,A2M,A2ML1,A4GALT,A4GNT,AAAS,AACS,AADAC,AADACL2,...,ZSWIM6,ZSWIM7,ZW10,ZWILCH,ZWINT,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1
5-fluorouracil,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
a-443654,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
a-770041,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
abt737,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
afatinib,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
wz4003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
xmd8-92,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zibotentan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zm447439,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
logger.info(f'The shape of cnv_allgenes_df {cnv_allgenes_df.shape}')
logger.info(f'The shape of methylation_allgenes_df {methylation_allgenes_df.shape}')
logger.info(f'The shape of mutation_allgenes_df {mut_allgenes_df.shape}')
logger.info(f'The shape of drug_target_allgenes_df {drug_target_allgenes_df.shape}')

logger.info('{} samples and {} drugs in {} records.'.format(
    len(set(response_df['CellLine'])), len(set(response_df['DrugName'])), len(response_df)))

genes = list(cnv_allgenes_df.columns)
response_array = response_df.values.tolist()

The shape of cnv_allgenes_df (634, 15059)
The shape of methylation_allgenes_df (708, 15059)
The shape of mutation_allgenes_df (937, 15059)
The shape of drug_target_allgenes_df (221, 15059)
479 samples and 219 drugs in 135353 records.


In [14]:
genes

['A1BG',
 'A1CF',
 'A2M',
 'A2ML1',
 'A4GALT',
 'A4GNT',
 'AAAS',
 'AACS',
 'AADAC',
 'AADACL2',
 'AADACL3',
 'AADACL4',
 'AADAT',
 'AAGAB',
 'AAK1',
 'AAMP',
 'AANAT',
 'AARS2',
 'AASDH',
 'AASDHPPT',
 'AASS',
 'AATF',
 'AATK',
 'ABAT',
 'ABCA1',
 'ABCA12',
 'ABCA13',
 'ABCA2',
 'ABCA3',
 'ABCA4',
 'ABCA5',
 'ABCA6',
 'ABCA7',
 'ABCA8',
 'ABCA9',
 'ABCB1',
 'ABCB10',
 'ABCB11',
 'ABCB4',
 'ABCB5',
 'ABCB6',
 'ABCB8',
 'ABCB9',
 'ABCC1',
 'ABCC10',
 'ABCC11',
 'ABCC12',
 'ABCC2',
 'ABCC3',
 'ABCC4',
 'ABCC5',
 'ABCC6',
 'ABCC8',
 'ABCC9',
 'ABCD2',
 'ABCD3',
 'ABCD4',
 'ABCE1',
 'ABCF1',
 'ABCF2',
 'ABCF3',
 'ABCG1',
 'ABCG2',
 'ABCG4',
 'ABCG5',
 'ABCG8',
 'ABHD1',
 'ABHD10',
 'ABHD11',
 'ABHD12',
 'ABHD12B',
 'ABHD13',
 'ABHD14A',
 'ABHD14B',
 'ABHD15',
 'ABHD2',
 'ABHD3',
 'ABHD4',
 'ABHD5',
 'ABHD6',
 'ABHD8',
 'ABI1',
 'ABI2',
 'ABI3',
 'ABI3BP',
 'ABL1',
 'ABL2',
 'ABLIM1',
 'ABLIM2',
 'ABLIM3',
 'ABO',
 'ABR',
 'ABRA',
 'ABT1',
 'ABTB1',
 'ABTB2',
 'ACAA1',
 'ACAA2',
 'ACACA',
 

In [15]:
'''
 第一列：Cell Line ID
 第二列：药物
 第三列：pIC50
'''
response_array

[['2313287', 've821', -2.323267],
 ['2313287', 'sb505124', -4.69609],
 ['2313287', 'nvp-adw742', -2.289179],
 ['2313287', 'lfm-a13', -4.194402],
 ['2313287', 'tipifarnib', -1.78317],
 ['2313287', 'bicalutamide', -4.69609],
 ['2313287', 'zstk474', -0.052341],
 ['2313287', 'entinostat', -2.205643],
 ['2313287', 'rvx-208', -4.242455],
 ['2313287', 'olaparib', -4.026938],
 ['2313287', 'embelin', -2.786412],
 ['2313287', 'azd8186', -0.495401],
 ['2313287', 'mk-1775', 0.256284],
 ['2313287', 'omipalisib', 4.022985],
 ['2313287', 'ruxolitinib', -4.15679],
 ['2313287', 'osi-027', -3.464966],
 ['2313287', 'phenformin', -8.524194],
 ['2313287', 'az960', -1.506737],
 ['2313287', 'gsk343', -2.581191],
 ['2313287', 'crizotinib', -4.191522],
 ['2313287', 'chir-99021', -3.333645],
 ['2313287', 'lestaurtinib', -1.372905],
 ['2313287', 'dabrafenib', -4.520531],
 ['2313287', 'alectinib', -4.121734],
 ['2313287', 'plx-4720', -4.398104],
 ['2313287', 'leflunomide', -4.743808],
 ['2313287', 'erlotinib', -1

## 训练集、验证集、测试集划分

In [16]:
seed = 1
split_path = Path(params['data_dir']).joinpath(data_folder_name, 'split')
if not exists(split_path):
    os.makedirs(split_path)
if not exists(split_path.joinpath('training_set_seed_'+str(seed)+'.csv')) or recreate == True:
    logger.info(f'Spliting the dataset with seed {seed}')
    sample_train, sample_test = train_test_split(response_df, test_size=0.2)
    sample_test, sample_valid = train_test_split(sample_test, test_size=0.5)
    sample_train.to_csv(split_path.joinpath('training_set_seed_'+str(seed)+'.csv'), index=False)
    sample_valid.to_csv(split_path.joinpath('valid_set_seed_'+str(seed)+'.csv'), index=False)
    sample_test.to_csv(split_path.joinpath('test_set_seed_'+str(seed)+'.csv'), index=False)
else:
    logger.info(f'Loading the split index with seed {seed}.')
    sample_train = pd.read_csv(split_path.joinpath('training_set_seed_'+str(seed)+'.csv'))
    sample_valid = pd.read_csv(split_path.joinpath('valid_set_seed_'+str(seed)+'.csv'))
    sample_test = pd.read_csv(split_path.joinpath('test_set_seed_'+str(seed)+'.csv'))

logger.info(f'{len(sample_train)} samples for train, {len(sample_valid)} samples for valid, {len(sample_test)} samples for test.\n')

respone_train_array = sample_train.values.tolist()
respone_valid_array = sample_valid.values.tolist()
respone_test_array = sample_test.values.tolist()

Spliting the dataset with seed 1
108282 samples for train, 13536 samples for valid, 13535 samples for test.



In [17]:
K = 5
Kfold_seed_path = data_path.joinpath('GDSC', 'Kfold-{}_seed-{}'.format(K, seed))
if not exists(Kfold_seed_path):
    os.makedirs(Kfold_seed_path)
if not exists(Kfold_seed_path.joinpath('training_K0.csv')) or recreate == True:
    kf = KFold(n_splits=K, shuffle=True, random_state=seed)
    for i, (train_index, test_index) in enumerate(kf.split(response_df)):
        sample_train = response_df.iloc[train_index]
        sample_test = response_df.iloc[test_index]
        sample_train.to_csv(Kfold_seed_path.joinpath('training_K{}.csv'.format(i)), index=False)
        sample_test.to_csv(Kfold_seed_path.joinpath('test_K{}.csv'.format(i)), index=False)
filename_prefix = ''